In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("ch2",set_enable=True)


LangSmith 추적을 시작합니다.
[프로젝트명]
ch2


### 프롬프트

- 언어 모델이 사용할 질문이나 명령을 생성하는 과정

### RAG 프롬프트 구조




In [3]:
"""
당신은 질문-답변(Question-Answer) Task 를 수행한는 AI 어시스턴트 입니다.
검색된 문맥(context)를 사용하여 질문(question)에 답하세요. 
만약, 문맥(context) 으로부터 답을 찾을 수 없다면 '모른다' 고 말하세요. 
한국어로 대답하세요.

#Question: 
{이곳에 사용자가 입력한 질문이 삽입됩니다}

#Context: 
{이곳에 검색된 정보가 삽입됩니다}
"""

"\n당신은 질문-답변(Question-Answer) Task 를 수행한는 AI 어시스턴트 입니다.\n검색된 문맥(context)를 사용하여 질문(question)에 답하세요. \n만약, 문맥(context) 으로부터 답을 찾을 수 없다면 '모른다' 고 말하세요. \n한국어로 대답하세요.\n\n#Question: \n{이곳에 사용자가 입력한 질문이 삽입됩니다}\n\n#Context: \n{이곳에 검색된 정보가 삽입됩니다}\n"

### 방법 1. from_template() 메소드를 사용하여 PromptTemplate 객체 생성

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name = "gpt-4o-mini")

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{country}의 수도는 어디야?")


In [8]:
chain = prompt|llm

chain.invoke('대한민국')

AIMessage(content='대한민국의 수도는 서울입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 15, 'total_tokens': 23, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f59a81427f', 'finish_reason': 'stop', 'logprobs': None}, id='run-7161376f-d22e-4f9f-a4ba-dbeefb69d6af-0', usage_metadata={'input_tokens': 15, 'output_tokens': 8, 'total_tokens': 23, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

### 방법 2. PromptTemplate 객체 생성과 동시에 prompt 생성

In [9]:
template = "{country}의 수도는 어디야?"

prompt = PromptTemplate(
    template=template,
    input_variables=["country"],
)

prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디야?')

In [10]:
prompt.format(country="대한민국")

'대한민국의 수도는 어디야?'

In [11]:
template = "{country1}와 {country2}의 수도는 어디야?"

prompt = PromptTemplate(
    template=template,
    input_variables=['country1'],
    partial_variables={
        "country2":"일본"
    }
)
prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '일본'}, template='{country1}와 {country2}의 수도는 어디야?')

In [12]:
prompt.format(country1="대한민국")

'대한민국와 일본의 수도는 어디야?'

In [15]:
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '캐나다'}, template='{country1}와 {country2}의 수도는 어디야?')

In [17]:
prompt_partial.format(country1="대한민국")

'대한민국와 캐나다의 수도는 어디야?'

In [18]:
chain = prompt_partial|llm

chain.invoke("대한민국")

AIMessage(content='대한민국의 수도는 서울이고, 캐나다의 수도는 오타와입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 18, 'total_tokens': 36, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-63280e90-f7ed-4be8-9554-e278f560f04e-0', usage_metadata={'input_tokens': 18, 'output_tokens': 18, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

### partial_variables: 부분 변수 채움

- 항상 공통된 방식으로 가져오고 싶은 변수 가 있는 경우

In [19]:
from datetime import datetime

# 오늘 날짜를 출력
datetime.now().strftime("%B %d")

'November 05'

In [20]:
def get_today():
    return datetime.now().strftime("%B %d")

In [21]:
prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today":get_today
    }
)

In [22]:
prompt.format(n=3)

'오늘의 날짜는 November 05 입니다. 오늘이 생일인 유명인 3명을 나열해 주세요. 생년월일을 표기해주세요.'

In [29]:
chain = prompt|llm
chain.invoke(3).content

'오늘, November 5일에 생일인 유명인 3명은 다음과 같습니다:\n\n1. **앤디 그리피스 (Andy Griffith)** - 1926년 6월 1일\n2. **레베카 로미진 (Rebecca Romijn)** - 1972년 11월 6일\n3. **다니엘 크레이그 (Daniel Craig)** - 1968년 3월 2일\n\n(참고로, 레베카 로미진은 11월 6일 생이지만, 오늘인 11월 5일에 생일이 다가오는 유명인으로 포함했습니다.) \n\n다른 유명인의 생일을 알고 싶으시면 말씀해 주세요!'

In [30]:
print(chain.invoke({"today": "Jan 02", "n": 3}).content)

오늘, 1월 2일에 생일인 유명인 3명은 다음과 같습니다:

1. **아이작 아시모프 (Isaac Asimov)** - 1920년 1월 2일
2. **J.R.R. 톨킨 (J.R.R. Tolkien)** - 1892년 1월 2일
3. **드류 배리모어 (Drew Barrymore)** - 1975년 1월 2일

이 외에도 많은 유명인들이 오늘 생일을 맞이하고 있습니다!


### 파일에서 프롬프트 불러오기

In [31]:
from langchain_core.prompts import load_prompt

prompt = load_prompt('prompts/fruit_color.yaml')
prompt

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit}의 색깔이 뭐야?')

In [34]:
prompt2 = load_prompt('prompts/capital.yaml')
print (prompt2.format(country = "대한민국"))

대한민국의 수도에 대해서 알려주세요.
수도의 특징을 다음의 양식에 맞게 정리해 주세요.
300자 내외로 작성해 주세요.
한글로 작성해 주세요.
----
[양식]
1. 면적
2. 인구
3. 역사적 장소
4. 특산품

#Answer:



### ChatPromptTemplate

- ChatPromptTemplate 은 대화목록을 프롬프트로 주입하고자 할 때 활용
- 메시지는 튜플(tuple) 형식으로 (role, message)로 구성 

In [35]:
from langchain_core.prompts import ChatPromptTemplate

### Role

- "system": 시스템 설정 메시지
- "human" : 사용자 입력 메시지
- "ai": AI 의 답변 메시지

In [37]:
chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는?")
print(chat_prompt)

input_variables=['country'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는?'), additional_kwargs={})]


In [39]:
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는?'

In [42]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"
)
messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 테디 입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='당신의 이름은 무엇입니까?', additional_kwargs={}, response_metadata={})]

In [43]:
llm.invoke(messages).content

'제 이름은 테디입니다! 당신은 어떻게 불리나요?'

In [44]:
chain = chat_template|llm
chain.invoke({'name':'teddy','user_input':"당신의 이름은 무엇인가요?"}).content

'제 이름은 Teddy입니다! 당신은 어떻게 되시나요?'

### MessagePlaceholder

- LangChain은 포맷하는 동안 렌더링할 메시지를 완전히 제어할 수 있는 MessagePlaceholder 를 제공

In [45]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [47]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)
chat_prompt


ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

In [48]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
)

print(formatted_chat_prompt)

System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.
Human: 안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.
AI: 반가워요! 앞으로 잘 부탁 드립니다.
Human: 지금까지의 대화를 5 단어로 요약합니다.


In [50]:
chain = chat_prompt | llm | StrOutputParser()
# chain 실행 및 결과확인
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
        ],
    }
)


'새로 입사한 테디, 반가움.'

In [ ]:
logging.langsmith("ch2",set_enable=False)

LangSmith 추적을 하지 않습니다.
